# Part I


#### Authors: Taniya Cai

## Introduction


## Population Parameters

## SRS

## Stratified Sampling

In comparison with a simple random sampling, we can divide the population into sub-populations, then apply random sampling of each subgroups to more accurately reflect the population studied based on a set of chosen characteristics.

* Assumptions: 

> All observations must be exactly under one and only one stratum.

> Sampling from each stratum must be random

* Hypothesis: 

> The value of the parameters chosen are the same across all stratum, that is , 1.) the proportion of personal income that exceeds $50K/yr is the same across all stratum, and 2.) the ........ is the same across all stratum.

## Analysis and Comparison

## Discussion 

## Conclusion 

# Part II